In [1]:
# imports
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense, Dropout, MaxPool2D
from keras.models import Sequential
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
import math
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
#### variabili globali
#### GIUSEPPE
#image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
image_path = '..\\..\\Untitled_Folder'
#### FABIO
# image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
csv_path = 'HAM10000_metadata.csv'
csv_completo_2 = 'dataframe_completo_2.csv'



In [3]:
# dal dataframe ne estrae le immagini in un array, serve nel batch
def load_images(array):
    images = list()
    #print(array.iloc[0])
    for index in range(0,len(array)):
        #print(index.type())
        img_id = array.iloc[index]
        elem = os.path.join(image_path, img_id)
        elem = elem + ".jpg"
        img = cv2.imread(elem)
        # TODO va fatto un reshape, non capisco perché
        images.append(np.asarray(img)) #########################################################################################
    return np.asarray(images)

def my_train_batch(dataset, batch, epochs, classifier):
    size = len(dataset)
    epoch=0
    cl1 = k.models.load_model(classifier)
    # train del classificatore un batch alla volta
    while size >= batch:
        # To get n random rows 
        samples = dataset.sample(n = batch)
        # splitting test and validation
        X_train, X_val, y_train, y_val = train_test_split(samples["image_id"], samples["dx"], test_size=0.30)
        
        train_img = np.asarray(load_images(X_train))
        val_img = np.asarray(load_images(X_val))
        
        #1hot encoder
        enc = OneHotEncoder(sparse=False)
        y_train_shaped = np.array(y_train).reshape(-1,1)
        y_val_shaped = np.array(y_val).reshape(-1,1)
        enc.fit(y_train_shaped)
        y_train_one = enc.transform(y_train_shaped)
        y_val_one = enc.transform(y_val_shaped)
        
        cl1.fit(train_img, y_train_one, epochs=epochs, shuffle=True, validation_data=(val_img, y_val_one), verbose=1)
        
        #pulizia memoria e reset per il prossimo ciclo
        del X_train, y_train_one, X_val, y_val_one, train_img, val_img
        dataset = dataset.drop(samples.index)
        size = len(dataset)
        del samples, enc
        epoch = epoch + 1
        print("fine epoca " + str(epoch) + ";\trestano " + str(size) + " campioni nel dataset")
        
    ######## ULTIMO CICLO CHE FINISCE IL DATASET
    samples = dataset.sample(n = size)
    # splitting test and validation
    X_train, X_val, y_train, y_val = train_test_split(samples["image_id"], samples["dx"], test_size=0.30)
    
    train_img = np.asarray(load_images(X_train))
    val_img = np.asarray(load_images(X_val))
    
    #1hot encoder
    enc = OneHotEncoder(sparse=False)
    y_train_shaped = np.array(y_train).reshape(-1,1)
    y_val_shaped = np.array(y_val).reshape(-1,1)
    enc.fit(y_train_shaped)
    y_train_one = enc.transform(y_train_shaped)
    y_val_one = enc.transform(y_val_shaped)
    
    cl1.fit(train_img, y_train_one, epochs=epochs, shuffle=True, validation_data=(val_img, y_val_one), verbose=1)
    
    del X_train, y_train_one, X_val, y_val_one, train_img, val_img
    dataset = dataset.drop(samples.index)
    size = len(dataset)
    del samples, enc
    epoch = epoch + 1
    print("fine ultima epoca")

    cl1.save(classifier)
    return

def kaggle_classifier():
    # Set the CNN model 
    # my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
    input_shape = (150, 200, 3)
    build_shape = (None, 150, 200, 3)
    num_classes = 7
    
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Conv2D(16, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(16,kernel_size=(3, 3), activation='relu',padding = 'Same',))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(32, (3, 3), activation='relu',padding = 'Same'))
    model.add(Conv2D(32, (3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.40))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.build(build_shape)
    model.summary()
    model.compile(optimizer='adadelta', loss='binary_crossentropy')
    return model

In [6]:
# creazione dataset piccolo
size = 3000

dataset = pd.read_csv(csv_completo_2, encoding = "ISO-8859-1")
new_data = pd.DataFrame()
for elem in range(7):
    print("starting class " + str(elem))
    samples = dataset[dataset["dx"] == elem].sample(n=size)
    new_data = new_data.append(samples)
new_data.to_csv('v02_dataframe_piccolo.csv', index=False)


starting class 0
starting class 1
starting class 2
starting class 3
starting class 4
starting class 5
starting class 6


In [7]:
# - dataset splittato 80 20
dataset = new_data

mass = int(len(dataset)*0.8)
train = dataset.sample(n=mass)
train.to_csv("v02_train08.csv", index=False)
test = dataset.drop(train.index)
test.to_csv("v02_test02.csv", index=False)

del test

In [4]:
train = pd.read_csv("v02_train08.csv", encoding = "ISO-8859-1")

In [9]:
# - creo cl_batch e cl_fit
cl_k = kaggle_classifier()
cl_k.save('v02_cl_kaggle_fit.h5')
cl_k.save('v02_cl_kaggle_batch.h5')


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 150, 200, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 200, 16)      448       
_________________________________________________________________
batch_normalization_2 (Batch (None, 150, 200, 16)      64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 200, 16)      2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 100, 16)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 75, 100, 16)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 100, 32)      

In [5]:
# - carico in memoria le immagini
# - suddivido in train validation
# - 1hot encoding delle y
images = load_images(train["image_id"])

X_train, X_val, y_train, y_val = train_test_split(images, train["dx"], test_size=0.10)

#1hot encoder
enc = OneHotEncoder(sparse=False)
y_train_shaped = np.array(y_train).reshape(-1,1)
y_val_shaped = np.array(y_val).reshape(-1,1)
enc.fit(y_train_shaped)
y_train_one = enc.transform(y_train_shaped)
y_val_one = enc.transform(y_val_shaped)

In [7]:
# kaggle fit
cl_name = "v02_cl_kaggle_fit.h5"
cl_k = k.models.load_model(cl_name)
cl_k.fit(X_train, y_train_one, epochs=50, shuffle=True, validation_data=(X_val, y_val_one), verbose=1)
cl_k.save(cl_name)

del X_train, X_val, y_train, y_val, y_train_shaped, y_val_shaped, y_train_one, y_val_one

Train on 15120 samples, validate on 1680 samples
Epoch 1/50
15120/15120 [==============================] - 581s 38ms/step - loss: 0.3538 - val_loss: 0.2974
Epoch 2/50
15120/15120 [==============================] - 638s 42ms/step - loss: 0.2934 - val_loss: 0.2657
Epoch 3/50
15120/15120 [==============================] - 585s 39ms/step - loss: 0.2732 - val_loss: 0.2566
Epoch 4/50
15120/15120 [==============================] - 583s 39ms/step - loss: 0.2541 - val_loss: 0.2418
Epoch 5/50
15120/15120 [==============================] - 577s 38ms/step - loss: 0.2395 - val_loss: 0.2186
Epoch 6/50
15120/15120 [==============================] - 647s 43ms/step - loss: 0.2228 - val_loss: 0.2490
Epoch 7/50
15120/15120 [==============================] - 598s 40ms/step - loss: 0.2082 - val_loss: 0.2131
Epoch 8/50
15120/15120 [==============================] - 607s 40ms/step - loss: 0.1934 - val_loss: 0.2178
Epoch 9/50
15120/15120 [==============================] - 607s 40ms/step - loss: 0.1798 - val_l

In [8]:
############ TEST
# - carico in memoria il classificatore
# - istanzio l'encoder e lo fitto

#dataset = pd.read_csv(csv_completo_2, encoding = "ISO-8859-1")
testset = pd.read_csv("v02_test02.csv", encoding = "ISO-8859-1")

enc = OneHotEncoder(sparse=False)
y_train_shaped = np.array(testset["dx"]).reshape(-1,1)
enc.fit(y_train_shaped)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)

In [9]:
# - carico in memoria le immagini test

test_img = load_images(testset["image_id"])

In [10]:
# - predict del classificatore k

predictions1 = cl_k.predict(test_img)
pred_reverse1 = enc.inverse_transform(predictions1)
print(sklearn.metrics.classification_report(testset["dx"], pred_reverse1))
print('F1-score: '+ str(round(sklearn.metrics.f1_score(testset["dx"], pred_reverse1, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.50      0.60      0.55       581
           1       0.62      0.57      0.59       585
           2       0.51      0.43      0.47       622
           3       0.62      0.82      0.70       588
           4       0.60      0.60      0.60       625
           5       0.83      0.75      0.79       595
           6       0.98      0.85      0.91       604

    accuracy                           0.66      4200
   macro avg       0.67      0.66      0.66      4200
weighted avg       0.67      0.66      0.66      4200

F1-score: 0.66


In [ ]:
# - FIT di cl_batch su train
cl_name = 'v02_cl_kaggle_batch.h5'
my_train_batch(dataset=train, batch=2000, epochs=10, classifier = cl_name)

In [ ]:
# - predict del kaggle batch
cl_k = k.models.load_model(cl_name)
predictions2 = cl_k.predict(test_img)
pred_reverse2 = enc.inverse_transform(predictions2)
print(sklearn.metrics.classification_report(testset["dx"], pred_reverse2))
print('F1-score: '+ str(round(sklearn.metrics.f1_score(testset["dx"], pred_reverse2, average='micro'),2)))